In [ ]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
# Network Parameters
n_hidden_1 = 10 # 1st layer number of features
n_hidden_2 = 10 # 2nd layer number of features
n_input = 10 # Number of feature
n_classes = 2 # Number of classes to predict

In [ ]:
# clean the graph
tf.reset_default_graph()

In [ ]:
# MLP
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with sigmoid activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define the cost/losss
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=pred)
    #xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)
    cost = tf.reduce_mean(xentropy, name='loss')

learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(cost)

# Evaluation
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# keras kind of MLP construct in TF (using layers.dense)
# Define the network
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

In [ ]:
# CNN
# This specifies the weights for either fully connected or convolutional layers 
# of the network. They are initialized randomly using a truncated normal distribution
# with a standard deviation of 0.1. This sort of initialization with a random normal
# distribution that is truncated at the tails is pretty common and generally produces
# good results
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    # initial = tf.random_normal()
    return tf.Variable(initial)
# This defines the bias elements in either fully connected or a convolutional layer.
# These are all initialized with the constant value of 0.1
def bias_variable(shape):
    initial = tf.Constant(0.1, shape=shape)
    return tf.Variable(initial)
# This specifies the convolution we will typically use. A full convolution (no skips)
# withh an output the same size as the input
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strindes=[1,1,1,1], padding='SAME')
# This sets the max pool to half the size across the height/width dimensions, and in 
# total a quarter the size of the feature map
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# This is the actual layer we will use. Linear convolution as defined in conv2d, with a bias,
# followed by te ReLU nonlinearity
def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable(shape)
    return tn.nn.relu(conv2d(input, W) + b)
# A standard full layer with a bias. Notice that here we didn't add the ReLU. This allows
# us to use the same layer for the final output, where we don't need the non-linar part.
def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

x_image = tf.reshape(x, [-1,28,28,1])
conv1 = conv_layer(x_image, shape=[5,5,1,32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_lauer(conv1_pool, shape=[5,5,32,64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full_drop = tf.nn.dropout(full_1, keep_prob= kepp_prob)

y_conv = full_layer(full_drop, 10)

with tf.name_scope('loss'):
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# RNN cells
# static unrolling through time
x0/x1 = tf.placeholder(tf.float32, [None, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [x0,x1],dtype=tf.float32)

# dynamic unrolling through time
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.nn.dynamic_rnn(basic_cell, x, dtype=tf.float32)

# multi-layer cells (statcking multiple RNNs)
multi_layer_cell = tf.contrib.rnn.MultiRnnCell(basic_cell)
output_seqs, states = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype=tf.float32)

# sequence classifier
prediction = tf.layers.dense(outputs[:, -1], 1, activation=tf.sigmoid)
softmax = tf.nn.softmax_cross_entropy-with_logits(lotits=final_output, labels=y)

# applying dropout
cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) for layer in range(n_layers)]
cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, inout_keep_prob=keep_prob) for cell in cells]
multi_layer_cell = tf.contrib.rnn.MultiRnnCell(cell_drop)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# Basic LSTM cell
lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
# peephole connections
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=True)

# GRU cell
gru_cell = tf.contrib.rnn.GRUCell(num_units=n_neurons)

# Embeddings
init_embeds = tf.random_uniform([vocabulary_size, embedding_size], -1.0,1.0)
embeddings = tf.Variable(init_embeds)
train_inputs = tf.placeholder(tf.float32, shape=[None])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# pre-trained embeddings
embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size, GLOVE_SIZE])
if PRE_TRAINED:
    embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, GLOVE_SIZE]), trainable=True)
    # if using pretrained embeddings, assign them to the embeddings variable
    embedding_init = embeddings.assign(embedding_placeholder)
    embed = tf.nn.embedding_lookup(embeddings, _inputs)
else:
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_dimension],-1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, _inputs)
    
# Bidirectional
with tf.name_scope('biGRU'):
    with tf.variable_scope('forward'):
        gru_fw_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
        gru_fw_cell = tf.contrib.rnn.DropoutWrapper(gru_fw_cell)
    with tf.variable_scope('backward'):
        gru_bw_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
        gru_bw_cell = tf.contrib.rnn.DropoutWrapper(gru_bw_cell)
        
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw=gru_fw_cell,
                                                     cell_bw=gru_bw_cell, input=embed, sequence_length=_seqlens,
                                                     dtype=tf.float32, scope='BiGRU')
    states = tf.concat(values=states, axis=1)